In [2]:
import random
import csv
import math

# -----------------------------
# Configuración global
# -----------------------------

# Rangos de nutrientes semanales (los que estamos usando ahora)
NUTRIENT_BOUNDS = {
    "Calories":      (14000.0, 17500.0),
    "Cholesterol":   (0.0,    2100.0),
    "Total_Fat":     (300.0,  650.0),
    "Sodium":        (0.0,    16000.0),
    "Carbohydrates": (1500.0, 2600.0),
    "Dietary_Fiber": (140.0,  280.0),
    "Protein":       (350.0,  700.0),
    "Vit_A":         (5000.0, 20000.0),
    "Vit_C":         (500.0,  7000.0),
    "Calcium":       (4900.0, 10500.0),
    "Iron":          (60.0,   280.0),
}

# Clasificación fija de cada alimento en el dataset pequeño
CATEGORY_MAP = {
    "Frozen Broccoli":        "veg",
    "Carrots Raw":            "veg",
    "Celery Raw":             "veg",
    "Frozen Corn":            "veg",
    "Lettuce Iceberg Raw":    "veg",
    "Peppers Sweet Raw":      "veg",
    "Tomato Red Ripe Raw":    "veg",

    "Potatoes Baked":         "carb",
    "Bagels":                 "carb",
    "Wheat Bread":            "carb",
    "White Bread":            "carb",
    "Cap'N Crunch":           "carb",
    "Cheerios":               "carb",
    "Corn Flakes Kellogg'S":  "carb",
    "Raisin Bran Kellogg'S":  "carb",
    "Rice Krispies":          "carb",
    "Special K":              "carb",
    "Oatmeal":                "carb",
    "Couscous":               "carb",
    "Macaroni cooked":        "carb",
    "White Rice":             "carb",

    "Apple Raw w/Skin":       "fruit",
    "Banana":                 "fruit",
    "Grapes":                 "fruit",
    "Kiwifruit Raw Fresh":    "fruit",
    "Oranges":                "fruit",

    "Tofu":                   "legume",
    "Peanut Butter":          "legume",

    "Butter Regular":         "dairy",
    "Cheddar Cheese":         "dairy",
    "Whole Milk":             "dairy",
    "Lowfat Milk":            "dairy",
    "Skim Milk":              "dairy",

    "Roasted Chicken":        "meat",
    "Poached Eggs":           "meat",
    "Scrambled Eggs":         "meat",
    "Bologna Turkey":         "meat",
    "Frankfurter Beef":       "meat",
    "Ham Sliced Extralean":   "meat",
    "Kielbasa Pork":          "meat",
    "Pork":                   "meat",

    "Sardines in Oil":        "fish",
    "White Tuna in Water":    "fish",

    "Pizza w/Pepperoni":      "prepared",
    "Hamburger w/Toppings":   "prepared",
    "Hotdog Plain":           "prepared",
    "Spaghetti W/ Sauce":     "prepared",

    "Oatmeal Cookies":        "sweet",
    "Apple Pie":              "sweet",
    "Chocolate Chip Cookies": "sweet",
    "Malt-O-Meal Choc":       "sweet",

    "Popcorn Air-Popped":     "snack",
    "Potato Chips":           "snack",
    "Pretzels":               "snack",
    "Tortilla Chips":         "snack",

    "Chicken Noodle Soup":    "soup",
    "Splt Pea&Ham Soup":      "soup",
    "Veggie Beef Soup":       "soup",
    "New Eng Clam Chwd":      "soup",
    "Tomato Soup":            "soup",
    "Crm Mshrm Soup":         "soup",
    "Bean Bacon Soup":        "soup",
}

# Restricciones semanales por categoría
CATEGORY_BOUNDS = {
    "veg":     (14, 28),   # 2–4 raciones de verdura/día
    "fruit":   (7, 21),    # 1–3 frutas/día
    "carb":    (7, 21),   # 1.5–4 raciones cereales/tubérculos/día
    "legume":  (2, 5),     # 2–5 raciones tofu+peanut butter/sem
    "dairy":   (5, 11),    # 3–10 lácteos/sem
    "meat":    (3, 7),     # proteína animal suficiente sin exceso
    "fish":    (3, 7),     # 2–7 pescados/sem

    "prepared": (0, 2),    # pizza, hamburguesa, hotdog, spaghetti w/sauce
    "snack":    (0, 2),    # snacks salados
    "sweet":    (0, 3),    # dulces/cereales azucarados
    "soup":     (0, 5),    # sopas procesadas
}

# Precios con aleatoriedad (ya afinados a España)
price_map = {
    "Lowfat Milk": 0.49,
    "Whole Milk": 0.44,
    "Apple Pie": 0.63,
    "Apple Raw w/Skin": 0.53,
    "Bagels": 0.37,
    "Banana": 0.42,
    "Bean Bacon Soup": 1.58,
    "Bologna Turkey": 0.52,
    "Butter Regular": 0.33,
    "Cap'N Crunch": 0.47,
    "Carrots Raw": 0.33,
    "Celery Raw": 0.28,
    "Cheddar Cheese": 0.86,
    "Cheerios": 0.42,
    "Chicken Noodle Soup": 1.63,
    "Chocolate Chip Cookies": 0.47,
    "Corn Flakes Kellogg'S": 0.43,
    "Couscous": 0.63,
    "Crm Mshrm Soup": 1.53,
    "Frankfurter Beef": 0.88,
    "Frozen Broccoli": 0.91,
    "Frozen Corn": 0.47,
    "Grapes": 0.53,
    "Ham Sliced Extralean": 0.63,
    "Hamburger w/Toppings": 4.17,
    "Hotdog Plain": 2.07,
    "Kielbasa Pork": 0.52,
    "Kiwifruit Raw Fresh": 0.52,
    "Lettuce Iceberg Raw": 0.32,
    "Macaroni cooked": 0.64,
    "Malt-O-Meal Choc": 1.01,
    "New Eng Clam Chwd": 1.57,
    "New Eng Clam Chwd w/Mlk": 1.51,  # por si lo usas
    "Oatmeal": 1.03,
    "Oatmeal Cookies": 0.42,
    "Oranges": 0.42,
    "Peanut Butter": 0.47,
    "Peppers Sweet Raw": 0.71,
    "Pizza w/Pepperoni": 2.05,
    "Poached Eggs": 0.38,
    "Popcorn Air-Popped": 0.38,
    "Pork": 1.38,
    "Potato Chips": 0.64,
    "Potatoes Baked": 0.48,
    "Pretzels": 0.52,
    "Raisin Bran Kellogg'S": 0.59,
    "Rice Krispies": 0.48,
    "Roasted Chicken": 3.82,
    "Sardines in Oil": 1.15,
    "Scrambled Eggs": 0.37,
    "Skim Milk": 0.48,
    "Spaghetti W/ Sauce": 2.54,
    "Special K": 0.48,
    "Splt Pea&Ham Soup": 1.63,
    "Tofu": 0.92,
    "Tomato Red Ripe Raw": 0.37,
    "Tomato Soup": 1.27,
    "Tortilla Chips": 0.63,
    "Veggie Beef Soup": 1.56,
    "Wheat Bread": 0.38,
    "White Bread": 0.33,
    "White Rice": 0.57,
    "White Tuna in Water": 1.77,
}

# -----------------------------
# Carga de datos
# -----------------------------

def load_foods(csv_file, nutrient_bounds):
    """
    Lee el CSV y devuelve una lista de alimentos con:
    - Name
    - Price (sobrescrito desde price_map)
    - Category (según CATEGORY_MAP)
    - nutrientes relevantes
    """
    foods = []
    with open(csv_file, newline='', encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            name = row["Name"]
            food = {
                "Name": name,
                "Price": float(price_map[name]),          # usamos nuestros precios
                "Category": CATEGORY_MAP[name],          # categoría fija
            }
            for nut in nutrient_bounds.keys():
                food[nut] = float(row[nut])
            foods.append(food)
    return foods


# -----------------------------
# Modelo del problema dietético
# -----------------------------

def compute_energy(state, foods, nutrient_bounds, lam=100.0):
    """
    Devuelve: energy_penalized, cost_real, totals_nutrients
    Penalización = nutrientes + categorías (lineal), ponderada por lam.
    """
    cost = 0.0
    totals = {nut: 0.0 for nut in nutrient_bounds.keys()}
    cat_totals = {cat: 0.0 for cat in CATEGORY_BOUNDS.keys()}

    for qty, food in zip(state, foods):
        if qty <= 0:
            continue
        cost += qty * food["Price"]
        for nut in nutrient_bounds.keys():
            totals[nut] += qty * food[nut]

        cat = food["Category"]
        if cat in cat_totals:
            cat_totals[cat] += qty

    # Penalización nutrientes (lineal)
    nut_penalty = 0.0
    for nut, (mn, mx) in nutrient_bounds.items():
        v = totals[nut]
        if v < mn:
            nut_penalty += (mn - v)
        elif v > mx:
            nut_penalty += (v - mx)

    # Penalización categorías (lineal)
    cat_penalty = 0.0
    for cat, (mn, mx) in CATEGORY_BOUNDS.items():
        v = cat_totals[cat]
        if v < mn:
            cat_penalty += (mn - v)
        elif v > mx:
            cat_penalty += (v - mx)

    penalty = nut_penalty + cat_penalty
    energy = cost + lam * penalty
    return energy, cost, totals


# -----------------------------
# Vecindario ENTERO
# -----------------------------

def get_neighbors_diet(state, max_portion=10, num_neighbors=30):
    """
    Genera vecinos alterando 1 o 2 alimentos en ±1 unidad.
    SOLO ENTEROS.
    """
    neighbors = []
    n = len(state)

    for _ in range(num_neighbors):
        neigh = state[:]  # copia
        k = 1 if random.random() < 0.7 else 2  # 70% 1 alimento, 30% 2 alimentos
        indices = random.sample(range(n), k)

        for i in indices:
            delta = random.choice([-1, 1])      # cambio entero
            new_val = neigh[i] + delta
            new_val = max(0, min(max_portion, new_val))
            neigh[i] = int(new_val)

        neighbors.append(neigh)

    return neighbors


def state_key(state):
    """Clave para lista Tabú (enteros ya no necesitan redondeos)"""
    return tuple(state)


# -----------------------------
# Tabu Search
# -----------------------------

def tabu_search_diet(foods, nutrient_bounds,
                     num_iterations=500,
                     tabu_size=100,
                     max_portion=10,
                     num_neighbors=30,
                     lam=100.0):
    n_foods = len(foods)

    # Solución inicial ENTERA
    current_solution = [random.randint(0, max_portion) for _ in range(n_foods)]
    current_energy, current_cost, _ = compute_energy(current_solution, foods, nutrient_bounds, lam)

    best_solution = current_solution[:]
    best_energy = current_energy
    best_cost = current_cost

    tabu_list = [state_key(current_solution)]

    for it in range(num_iterations):
        neighbors = get_neighbors_diet(current_solution, max_portion, num_neighbors)

        best_neighbor = None
        best_neighbor_energy = math.inf

        for neigh in neighbors:
            key = state_key(neigh)
            if key in tabu_list:
                continue
            e, c, totals = compute_energy(neigh, foods, nutrient_bounds, lam)
            if e < best_neighbor_energy:
                best_neighbor = neigh
                best_neighbor_energy = e
                best_neighbor_cost = c
                best_neighbor_totals = totals

        if best_neighbor is None:
            tabu_list = [state_key(current_solution)]
            continue

        # Avanzar
        current_solution = best_neighbor
        current_energy = best_neighbor_energy
        current_cost = best_neighbor_cost

        # Tabú update
        tabu_list.append(state_key(current_solution))
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Mejor global
        if current_energy < best_energy:
            best_solution = current_solution[:]
            best_energy = current_energy
            best_cost = current_cost

        if it % 50 == 0:
            print(f"Iter {it}: energy={best_energy:.4f}, cost={best_cost:.4f}")

    _, _, best_totals = compute_energy(best_solution, foods, nutrient_bounds, lam)
    return best_solution, best_energy, best_cost, best_totals


# -----------------------------
# Main
# -----------------------------

if __name__ == "__main__":
    random.seed(10)

    nutrient_bounds = NUTRIENT_BOUNDS

    foods = load_foods("diet_with_prices.csv", nutrient_bounds)
    print(f"{len(foods)} alimentos cargados.")

    best_state, best_energy, best_cost, best_totals = tabu_search_diet(
        foods,
        nutrient_bounds,
        num_iterations=2000,
        tabu_size=200,
        max_portion=7,
        num_neighbors=100,
        lam=100  # penalización moderada: ajusta si quieres más dureza en restricciones
    )

    print("\n📌 Mejor energy (coste penalizado):", best_energy)
    print("💰 Coste real (sin penalización):", best_cost)

    print("\n🔍 Nutrientes totales:")
    for nut, (mn, mx) in nutrient_bounds.items():
        v = best_totals[nut]
        print(f"  {nut}: {v:.2f} (mín {mn} – máx {mx})")

    # Cálculo de totales por categoría para inspección
    cat_totals = {cat: 0.0 for cat in CATEGORY_BOUNDS.keys()}
    for qty, food in zip(best_state, foods):
        if qty <= 0:
            continue
        cat_totals[food["Category"]] += qty

    print("\n🥗 Raciones por categoría:")
    for cat, (mn, mx) in CATEGORY_BOUNDS.items():
        print(f"  {cat}: {cat_totals[cat]:.2f}  (mín {mn} – máx {mx})")

    print("\n🥗 Alimentos en la dieta:")
    for qty, food in zip(best_state, foods):
        if qty > 0:
            print(f"  {food['Name']}: {qty} porciones")

62 alimentos cargados.
Iter 0: energy=19233409.0500, cost=199.0500
Iter 50: energy=689139.0400, cost=119.0400
Iter 100: energy=2991.6300, cost=91.6300
Iter 150: energy=1383.0800, cost=83.0800
Iter 200: energy=469.0700, cost=69.0700
Iter 250: energy=367.3600, cost=67.3600
Iter 300: energy=164.3500, cost=64.3500
Iter 350: energy=163.2800, cost=63.2800
Iter 400: energy=162.5400, cost=62.5400
Iter 450: energy=162.5400, cost=62.5400
Iter 500: energy=62.2200, cost=62.2200
Iter 550: energy=61.3400, cost=61.3400
Iter 600: energy=60.2200, cost=60.2200
Iter 650: energy=59.5500, cost=59.5500
Iter 700: energy=59.2700, cost=59.2700
Iter 750: energy=59.2700, cost=59.2700
Iter 800: energy=59.2700, cost=59.2700
Iter 850: energy=58.7900, cost=58.7900
Iter 900: energy=58.4500, cost=58.4500
Iter 950: energy=57.3900, cost=57.3900
Iter 1000: energy=57.2600, cost=57.2600
Iter 1050: energy=57.2600, cost=57.2600
Iter 1100: energy=57.2600, cost=57.2600
Iter 1150: energy=57.2600, cost=57.2600
Iter 1200: energy=